# Inputs

In [1]:
input_masks_dir = '/Users/guillemcobos/Downloads/project_colony_counting-2023_06_03_02_59_55-segmentation mask 1.1/SegmentationClass'

In [2]:
data_as_cvat_tasks_dir = '/Users/guillemcobos/Library/CloudStorage/GoogleDrive-guillem.cobos@koabiotech.com/.shortcut-targets-by-id/1UQ_YnLRmoAqhCue-qXYNP9QhYorsiAxS/KOA Biotech/03. KOA DATA/Colony counting/colony-counting-yolov8/data.as.cvat.tasks/#177088 AAAAA'

In [3]:
only_annotated_images = True

# Code

In [4]:
from imageannot_multilabel import create_yolo_annotations
from sklearn.model_selection import train_test_split
import shutil
import os

# This will create .txt annotations based on the segmentation masks at input_masks_dir
create_yolo_annotations(input_masks_dir)

# Copy task directory if it doesn't exist
data_as_cvat_tasks_dir_copy = '/Users/guillemcobos/Library/CloudStorage/GoogleDrive-guillem.cobos@koabiotech.com/.shortcut-targets-by-id/1UQ_YnLRmoAqhCue-qXYNP9QhYorsiAxS/KOA Biotech/03. KOA DATA/Colony counting/colony-counting-yolov8/data.as.cvat.tasks/#177088 AAAAA copy'

if not(os.path.exists(data_as_cvat_tasks_dir_copy)):
    shutil.copytree(data_as_cvat_tasks_dir, data_as_cvat_tasks_dir_copy)

# Copy those annotations into data_as_cvat_tasks_dir_copy
annotation_files = [file for file in os.listdir(input_masks_dir) if file.endswith('.txt')]

for file in annotation_files:
    shutil.copy2(os.path.join(input_masks_dir, file), os.path.join(data_as_cvat_tasks_dir_copy, file))

# Create empty .txt files for those images in data_as_cvat_tasks_dir_copy with no annotations
files = os.listdir(data_as_cvat_tasks_dir_copy)
pics = [os.path.splitext(file)[0] for file in files if file.endswith(('.jpeg', '.png', '.jpg'))]
txts = [os.path.splitext(file)[0] for file in files if file.endswith('.txt')]
txts = list(set(txts).intersection(pics))
pics_with_no_annotations = list(set(pics) - set(txts))
for file in pics_with_no_annotations:
    file_path = os.path.join(data_as_cvat_tasks_dir_copy, file +'.txt')
    # Create an empty text file
    with open(file_path, 'w') as file:
        pass

if only_annotated_images == True:
    # Execute the following block if training is to be done exclusively over
    files = os.listdir(data_as_cvat_tasks_dir_copy)
    pics = sorted([file for file in files if file.endswith(('.jpeg', '.png', '.jpg'))])
    txts = sorted([file for file in files if file.endswith('.txt')])
    annotated = list(set([os.path.splitext(pic)[0] for pic in pics]).intersection([os.path.splitext(txt)[0] for txt in txts if os.path.getsize(os.path.join(data_as_cvat_tasks_dir_copy, txt))!=0]))
    pics = [name + '.jpeg' for name in annotated]
    txts = [name + '.txt' for name in annotated]

print(len(pics), len(txts))


# From data.as.cvat.task directory to *yolo official data* directory
# Splitting the data into training and testing sets
pics_train, pics_val, labels_train, labels_val = train_test_split(pics, txts, test_size=0.2, random_state=20)

for file in pics_train:
    source_path = os.path.join(data_as_cvat_tasks_dir_copy, file)
    destination_path = os.path.join(data_as_cvat_tasks_dir_copy + '/data/images/train/', file)
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    shutil.copy2(source_path, destination_path)

for file in pics_val:
    source_path = os.path.join(data_as_cvat_tasks_dir_copy, file)
    destination_path = os.path.join(data_as_cvat_tasks_dir_copy + '/data/images/val/', file)
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    shutil.copy2(source_path, destination_path)

for file in labels_train:
    source_path = os.path.join(data_as_cvat_tasks_dir_copy, file)
    destination_path = os.path.join(data_as_cvat_tasks_dir_copy + '/data/labels/train/', file)
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    shutil.copy2(source_path, destination_path)

for file in labels_val:
    source_path = os.path.join(data_as_cvat_tasks_dir_copy, file)
    destination_path = os.path.join(data_as_cvat_tasks_dir_copy + '/data/labels/val/', file)
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    shutil.copy2(source_path, destination_path)

63 63
